# 3D U^2-Net: A 3D Universal U-Net for Multi-Domain Medical Image Segmentation:
Test things here
https://theaisummer.com/unet-architectures/

## Imports

## U^2-net blocks 

## Prepare Dataset

### Load Data

### Create DataLoader

## Train Model

## Test Model